# Langchain and RAG Testing

Initialize the model

In [1]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/Users/zaidsaheb/Desktop/projects/LLMsalesbot/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Asking something in langchain

In [2]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand what love is.")
]

In [3]:
res = chat(messages)
print(res.content)

/Users/zaidsaheb/Desktop/projects/LLMsalesbot/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Love is a complex and multifaceted emotion that can be experienced in many different ways. It can involve feelings of affection, attachment, care, and concern for another person. Love can manifest in various forms, such as romantic love, platonic love, familial love, or even love for oneself. It often involves a deep connection and understanding between individuals, as well as a desire to support and nurture each other. Love can bring joy, fulfillment, and a sense of belonging, but it can also be challenging and require effort and compromise to maintain. Overall, love is a powerful and transformative force that plays a central role in human relationships and well-being.


To maintain context, you need to append extra messages to the previous chain

In [4]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Do you believe in love?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

As an AI assistant, I don't have personal beliefs or emotions like humans do. However, I can provide information and insights on the concept of love based on data and knowledge. Love is a common and universal experience for many people, and it plays a significant role in shaping human relationships and connections.


GPT 3.5 is not supposed to have hallucinations, I'm going to ask it something it knows nothing about to test this

In [5]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What do you know about my classmate Ismael Picazo?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [18]:
print(res.content)

I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me during our conversation. If you have any specific questions or topics you'd like to discuss, feel free to let me know.


Lets add some more context and tell it who Ismael is

In [11]:
llmchain_information = [
    "Ismael Picazo is a student at the IE University, great guy.",
    "Ismael is known for his work in the field of computer science.",
    "Ismael has a great future ahead of him."
]

source_knowledge = "\n".join(llmchain_information)

Now we feed it the information just like this

In [12]:
query = "Can you tell me about Ismael Picazo?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [13]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [14]:
print(res.content)

Ismael Picazo is a student at the IE University known for his work in the field of computer science. He is described as a great guy with a promising future ahead of him.


Importing data

In [10]:
import json
with open('EV_data.json', 'r') as file:
    product_data = json.load(file)


In [16]:
from langchain.prompts import ChatPromptTemplate

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embedding_function = OpenAIEmbeddings()

loader = JSONLoader(file_path="./EV_data.json", jq_schema=".electric_vehicles[]", text_content=False)
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

query = "What is the seating capacity of the Thunderbolt S?"
print(chain.invoke(query))

The seating capacity of the Thunderbolt S is 4.
